In [1]:
# Import the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import keras
from keras.models import Sequential
from keras.layers import Dense

import matplotlib.pyplot as plt

In [2]:
# Download data and split training & test sets 
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Split columns into Predictors and Target
concrete_data_columns = concrete_data.columns

P = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
T = concrete_data['Strength'] # Strength column

In [4]:
# Mormalise the Predictors
P_norm = (P - P.mean()) / P.std()
P_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [5]:
# number of predictors
n_cols = P_norm.shape[1] 

In [6]:
# Define the regression model with three hidden layers
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Build the model
model = regression_model()

In [7]:
# We create a 50-cycle training & evaluation loop. We would train the model on a 50-epoch fit -per cycle- and evaluate using MSE per cycle. We record each cycle MSE into a list.

score_mse_list = [] # Create a list to record the MSEs per cycle  
cycle  = 0 # Initialise the cycles

for looping in range(50): # Loop 50 times
    
    # Split the Predictors & Target data, holding 30% for testing
    P_norm_train, P_norm_test, T_train, T_test = train_test_split(P_norm, T,
                                                    test_size = 0.3,
                                                    random_state = 1)
    #Initialise the cycle MSE
    score_mse = 0
    
    # Fit the model 
    model.fit( P_norm_train, T_train, epochs=50, verbose=2)
    
    # Generate predicted test target values
    T_test_hat = model.predict(P_norm_test)
    
    # Cycle evaluation using scikit-learn mean_squared_error
    score_mse = mean_squared_error(T_test, T_test_hat) 
    
    # Append the loop MSE on the list
    score_mse_list.append(score_mse)

Epoch 1/50
23/23 - 0s - loss: 1583.3091 - 357ms/epoch - 16ms/step
Epoch 2/50
23/23 - 0s - loss: 1558.9574 - 20ms/epoch - 855us/step
Epoch 3/50
23/23 - 0s - loss: 1539.2307 - 19ms/epoch - 832us/step
Epoch 4/50
23/23 - 0s - loss: 1515.0707 - 19ms/epoch - 829us/step
Epoch 5/50
23/23 - 0s - loss: 1476.3252 - 19ms/epoch - 826us/step
Epoch 6/50
23/23 - 0s - loss: 1414.8770 - 19ms/epoch - 833us/step
Epoch 7/50
23/23 - 0s - loss: 1316.7599 - 19ms/epoch - 828us/step
Epoch 8/50
23/23 - 0s - loss: 1169.7638 - 18ms/epoch - 790us/step
Epoch 9/50
23/23 - 0s - loss: 970.8862 - 19ms/epoch - 809us/step
Epoch 10/50
23/23 - 0s - loss: 736.0144 - 19ms/epoch - 829us/step
Epoch 11/50
23/23 - 0s - loss: 513.0952 - 19ms/epoch - 829us/step
Epoch 12/50
23/23 - 0s - loss: 362.1767 - 20ms/epoch - 876us/step
Epoch 13/50
23/23 - 0s - loss: 291.9079 - 19ms/epoch - 829us/step
Epoch 14/50
23/23 - 0s - loss: 258.1655 - 19ms/epoch - 829us/step
Epoch 15/50
23/23 - 0s - loss: 236.6885 - 18ms/epoch - 783us/step
Epoch 16/50

In [8]:
# Printing the list of MSEs and verify the its length
print(score_mse_list)
len(score_mse_list)

[145.62188622314662, 113.51173758182651, 79.99325057718123, 45.31837986531871, 40.517568410717395, 38.386024283390775, 37.8471795463371, 39.20836549758309, 38.342790785604414, 38.42367990918137, 38.770916686490686, 38.87245476259034, 39.10786461518292, 39.42874679115767, 39.258864607586986, 42.42618742654424, 40.26979227531096, 40.102827647244276, 40.20669399058623, 40.473064361719125, 40.429656049558325, 40.88990920429894, 40.99491727113599, 41.04717940242227, 41.282951028711246, 40.902552268277965, 40.94814934952287, 40.91529430285838, 40.899653972333084, 40.74638781104583, 40.475530341215304, 39.89708973505703, 40.0130673273034, 39.68535404329272, 39.96721127527216, 40.48530905818599, 38.99915952868242, 38.70398768002911, 38.53627673631286, 38.65689136282187, 38.31098736901142, 38.5830998023237, 38.88946133422455, 38.2888172507932, 38.14052502544633, 39.513155812174034, 38.28799849050292, 37.65311035037301, 38.38593866814884, 37.76156824309944]


50

In [9]:
# Calculate and report the mean & std of the MSEs from the list
print('The training values mean of the 50-cycle MSEs is :', np.mean(score_mse_list))
print('The training values standard deviation of the 50-cycle MSEs is :', np.std(score_mse_list))

The training values mean of the 50-cycle MSEs is : 44.08758931878272
The training values standard deviation of the 50-cycle MSEs is : 18.665978399516675


## The mean and the standard deviation of the MSEs have significantly decreased by using a 3-hidden layer model, giving us the optimal results so far.